# 3 - Easy Wins (2 / 2)
##### **Author: Adam Gatt**

## explicit (pre 11)

Here we have a class representing a scaling factor that you can multiply other numbers with.

In [1]:
class Factor {
    public:
    Factor(double value)
        : value(value) { }
    
    double apply(double input) const {
        return input * value;
    }
    
    private:
    double value;
};

In [2]:
double scale(double input, const Factor& factor) {
    return factor.apply(input);
};

In [3]:
scale(5, 0.5)

2.5000000

How come the call to `scale()` succeeded even though the second parameter was a double instead of a Factor? It's because of the rules C++ uses when parsing a function call. If the compiler can't find a function declaration that matches the called signature, then it will attempt a process called _implicit conversion_. This involves looking around for any other functions that it can use to convert the arguments to types that will allow the original function call to succeed. It is similar to the promotion of primitives, so that you can pass an int to a function that expects a double, except broadened to apply to other types as well. The compiler will search for any user-defined conversion functions and constructors that it can use to perform the conversion process. 

So here:
1. The call to `scale(int, double)` is initially unsuccessful as that signature has not been declared.
2. The compiler notices the declaration `scale(int, const Factor&)` and determines if it can convert the arguments to match it.
3. The compiler notices that `Factor(double value)` is available for creating a Factor out of a double.
4. The compiler inserts the implicit conversion, effectively turning the function call into `scale(5, Factor(0.5))`, and creating a temporary Factor object to submit to the scale function.

C++ performs implicit conversion not only when calling functions, [but in a range of scenarios](https://en.cppreference.com/w/cpp/language/implicit_conversion) where one type is provided where another is needed.

Initially this might seem like a benefit, for getting code to work that otherwise wouldn't. But remember that this all happens implicitly behind the scenes, without evidence in the source code itself. As a general rule, you want your application logic to be explicitly laid out to make the code more readable and verifiable.

What are the costs or risks of implicit conversion? They might include:
 * Function arguments being open to a far greater range of types than intended
 * An API that is open to guesswork rather than understanding proper usage
 * Creation of unintended temporary objects
 * Creation of objects at all that shouldn't be allowed (e.g. if the class is intended Singleton)
 * Reducing clarity of the intent of the codebase
 
How do we crack down on implicit conversion, assuming we intend to? Remember that the implicit conversion comes about due to three factors:
1. A function call with a signature that hasn't been declared
2. A candidate signature that could be matched with some conversion
3. A conversion function or constructor that can perform the conversion

Here the `explicit` keyword is provided to solve our problem by addressing factor 3. A conversion function or constructor that is marked explicit will only be used by the compiler if explicitly called in the codebase. It will not be considered during the implicit conversion process as a way of performing a needed conversion.

In [4]:
class Factor2 {
    public:
    // Now explicit, no type conversion for arguments
    explicit Factor2(int value)
        : value(value) { }
    
    int apply(int input) const {
        return input * value;
    }
    
    private:
    int value;
}

In [5]:
double scale2(double input, const Factor2& factor) {
    return factor.apply(input);
};

In [6]:
scale2(5, 0.5)

input_line_15:2:2: error: no matching function for call to 'scale2'
 scale2(5, 0.5)
 ^~~~~~
input_line_14:1:8: note: candidate function not viable: no known conversion from 'double' to 'const __cling_N56::Factor2' for 2nd argument
double scale2(double input, const Factor2& factor) {
       ^


Interpreter Error: 

Here is another use for `explicit`. In the previous notebook we had `Line myLine = NULL` (missing the `*`) and the code not only compiled, but called the constructor `Line (int thickness)`. What happened here?

The issue is that the Line class, like our original Factor class, has a constructor that the compiler considers to be a [converting constructor](https://en.cppreference.com/w/cpp/language/converting_constructor). Pre C++11 this was any constructor that accepts only a single parameter. For '11 and later, any constructor is treated as a converting constructor (due to the introduction of initializer lists).

The example below demonstrates this. The important point to know here is that despite the `=` character, the operation below is not an assignment. Instead it follows the _copy initialisation_ process, which will end up calling the constructor. _(The next section will touch briefly on copy initialisation vs direct initialisation)._

In [7]:
Factor myFactor = 0.5; // Compiles to "Factor myFactor(0.5)"
myFactor.apply(10)

5.0000000

Declaring a constructor `explicit` will prevent it from being a converting constructor, and the statement returns to attempting to assign a double to our user-defined type.

In [8]:
Factor2 myFactor2 = 0.5;
myFactor2.apply(100)

input_line_18:2:10: error: no viable conversion from 'double' to '__cling_N56::Factor2'
 Factor2 myFactor2 = 0.5;
         ^           ~~~
input_line_13:1:7: note: candidate constructor (the implicit copy constructor) not viable: no known conversion from 'double' to 'const __cling_N56::Factor2 &' for 1st argument
class Factor2 {
      ^
input_line_13:1:7: note: candidate constructor (the implicit move constructor) not viable: no known conversion from 'double' to '__cling_N56::Factor2 &&' for 1st argument


Interpreter Error: 

For extra reading, here is a 2006 (pre-11) article on this topic on [The Old New Thing](https://devblogs.microsoft.com/oldnewthing/20060524-12/?p=31083).

## Uniform Initialisation Syntax

The simple initialisation of a variable might be one of the most complicated things in C++. Firstly, there is a bewildering number of ways that a variable can actually be initialised, [highlighted by this joke](http://mikelui.io/img/c++_init_forest.gif). I don't even know what the majority of these mechanisms are. More than this, the different methods of initialisation come with different rules and levels of strictness. [This is a very useful chart](https://josuttis.com/cpp/c++initialization.pdf) to help make the differences easy to compare. On that chart, the notable entries appear to be those shown with red braces. The `Type i{x};` row (and the related `Type i = {x};`) seem to have the advantage for pretty much all of the listed criteria. This makes it a strong choice if those criteria seem useful or important to you, such as enforcing that narrowing operations should result in errors.

In [9]:
double totalFunds = 10.0;

int x(totalFunds / 4); // Some compilers warn, but all will still allow
int y{totalFunds / 4}; // Narrowing is error

input_line_19:4:7: error: type 'double' cannot be narrowed to 'int' in initializer list [-Wc++11-narrowing]
int y{totalFunds / 4}; // Narrowing is error
      ^~~~~~~~~~~~~~
input_line_19:4:7: note: insert an explicit cast to silence this issue
int y{totalFunds / 4}; // Narrowing is error
      ^~~~~~~~~~~~~~
      static_cast<int>( )


Interpreter Error: 

The curly-braced approach is the [Uniform Initialisation Syntax](https://www.learncpp.com/cpp-tutorial/b-4-initializer-lists-and-uniform-initialization/) introduced with C++11 to make initialisation safer and more predictable. It comes with a new syntax, using braces, to avoid stepping on the toes of existing initialisation which must be kept unchanged for compatibility reasons.

The contents of the braces is known as an _initialiser list_ and is a powerful new tool for easily initialising complex structures. Firstly we can use it when instantiating objects and structs, just like with curved brackets:

In [10]:
struct Employee {
    std::string name;
    int age;
    double performanceRating;
};

// Initialiser list populates members in order of their declaration
Employee newHire{"Adam", 34, 4.0};
newHire.name

"Adam"

More powerfully, we can initialise vectors and arrays all at once. We can now represent the "list literal".

In [11]:
#include <vector>
#include <string>

std::vector<std::string> colours{"Red", "Yellow", "Orange", "White"};

// May I never initialise a vector like this again, so help me God
std::vector<std::string> birds;
birds.push_back("Magpie");
birds.push_back("Ostrich");
birds.push_back("Goose");
birds.push_back("Albatross");
birds.push_back("Hummingbird");

In [12]:
#include <array>
#include <set>
#include <utility>

// New-style array
std::array<int, 6> lotteryNumbers{4, 8, 15, 16, 23, 42};

// C-style array, dynamic-memory
int* okayTheyreTheLostNumbers = new int[6]{4, 8, 15, 16, 23, 42};

// Sets, pairs
std::set<char> typedKeys{'g', 'q', 'v', 'o', 'q', 'm', 't'};
std::pair<std::string, float> population{"South Australia", 1.7e6};

And with nested braces we can initialise complex data types:

In [13]:
#include <unordered_map>
#include <iostream>

std::unordered_map<std::string, std::string> capitols{
    {"Malta", "Valetta"},
    {"New Zealand", "Wellington"},
    {"United States", "Washington D.C"},
    {"Australia", "Canberra"},
    {"New Caledonia", "Nouméa"}
};

std::cout << "The capitol of Malta is " << capitols["Malta"] << std::endl;

The capitol of Malta is Valetta


And even nested structures:

In [14]:
#include <iostream>
#include <string>
#include <list>
#include <algorithm>

struct Runtime {
    std::string name;
    std::list<std::string> languages;
    int developmentYear;
};

// Second item is itself a full initialiser list, used for initialising the second declared member
Runtime jvm{"Java Virtual Machine", {"Java", "Kotlin", "Scala", "Clojure"}, 1994};

std::cout << jvm.name << " supports at least " << jvm.languages.size() << " languages";

Java Virtual Machine supports at least 4 languages

### But a gotcha, initialiser lists make uniform initialisation not fully uniform
Initialiser lists are supported by special intialiser-list constructors, which are provided for most container types. According to the standard, these constructors are attempted before other constructors. This can be a problem when trying to call regular constructors with argument types that match the container's template, for example:
* `vector( size_type count )` for vector\<int\>
* `basic_string( size_type count, CharT ch, const Allocator& alloc = Allocator());`

In [15]:
// Prepare a vector of 100 ints according to the std::vector(int size) constructor
std::vector<int> raffleDraws{100};

// Oh, actually it treated this as a single-element initialiser list
std::cout << raffleDraws.size() << ": " << raffleDraws[0];

1: 100

In [16]:
// But this container of strings will actually create a 100-length vector, now that
// it is unable to call the initialiser-list constructor with a 1-length list of ints

std::vector<std::string> attendees{100};
std::cout << attendees.size() << ": " << attendees[0];

100: 

In [17]:
// I want to use the constructor to prefill a string of 100 '_' characters
std::string underline{100, '_'};

underline

"d_"

In [18]:
// But returning to regular bracket initialisation works
std::string underline(100, '_');

underline

"____________________________________________________________________________________________________"

### So should I use or avoid braces initialisation?

Scott Meyers in Effective Modern C++:

> Most developers end up choosing one kind of delimiter as a default, using the other only when they have to. Braces-by-default folks are attracted by their unrivaled breadth of applicability, their prohibition of narrowing conversions, and their immunity to C++’s most vexing parse. Such folks understand that in some cases (e.g., creation of a std::vector with a given size and initial element value), parentheses are required. On the other hand, the go-parentheses-go crowd embraces parentheses as their default argument delimiter. They’re attracted to its consistency with the C++98 syntactic tradition, its avoidance of the auto-deduced-a-std::initializer_list problem, and the knowledge that their object creation calls won’t be inadvertently waylaid by std::initializer_list constructors. They concede that sometimes only braces will do (e.g., when creating a container with particular values). There’s no consensus that either approach is better than the other, so my advice is to pick one and apply it consistently.

## auto (in some situations)

## Newer cast syntax